In [ ]:
import delfi.distribution as dd
import delfi.generator as dg
import delfi.inference as infer
import delfi.summarystats as ds
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

from delfi.simulator.Gauss import Gauss


n_params = 1
seed = 42
m = Gauss(dim=n_params, seed=seed, noise_cov=1.)
p = dd.Gaussian(m=0. * np.ones(n_params), 
                S=1. *np.eye(n_params),
               seed=seed)
s = ds.Identity()
g = dg.Default(model=m, prior=p, summary=s)

# generate and plot
params, stats = g.gen(250)
plt.plot(stats, params, 'b.');
plt.xlabel('x')
plt.ylabel('theta')

obs = 0.8 * np.ones((1,1)) #_,obs = g.gen(1)


In [ ]:
# set up inference
n_fits = 20
out_snpe   = np.zeros((n_fits,2))
out_cdelfi = np.zeros((n_fits,2))
for idx_seed in range(n_fits):
    
    seed = idx_seed + 42

    print('seed: ', seed)
    for algo in range(2):
        m = Gauss(dim=n_params, seed=seed, noise_cov=1.)
        p = dd.Gaussian(m=0. * np.ones(n_params), 
                        S=1. * np.eye(n_params),
                       seed=seed)
        s = ds.Identity()
        g = dg.Default(model=m, prior=p, summary=s)
        
        if algo==0:
            res = infer.CDELFI(g, n_hiddens=[10], seed=seed, svi=True, obs=obs)
        elif algo==1:
            res = infer.SNPE(g, n_hiddens=[10], seed=seed, svi=True, obs=obs)

        # run with N samples
        log, trn_data = res.run(5000, n_rounds=5)
        posterior = res.predict(np.array([obs]).reshape(1,-1))

        
        if algo==0:
            out_cdelfi[idx_seed,:] = (posterior.mean, posterior.std**2)
            print('- CDELFI: \n', out_cdelfi[idx_seed,:])
        elif algo==1:
            out_snpe[idx_seed,:] = (posterior.mean, posterior.std**2)
            print('- SNPE: \n', out_snpe[idx_seed,:])
        


In [ ]:
mu_true = 0.4
sig_true = 0.5

plt.figure(figsize=(12,9))

plt.subplot(2,2,1)
plt.hist(out_cdelfi[:,0], bins = np.linspace(0.3, 0.55, 20))
plt.plot(mu_true*np.ones(2), [0, 14], 'r', linewidth=2)
plt.ylabel('counts')
plt.title('CDELFI')
plt.legend(['analytic solution','outcome (n=50)'], frameon=False)
plt.subplot(2,2,2)
plt.hist(out_cdelfi[:,1], bins = np.linspace(0.4, 0.65, 20))
plt.plot(sig_true*np.ones(2), [0, 14], 'r', linewidth=2)
plt.title('CDELFI')

plt.subplot(2,2,3)
plt.hist(out_snpe[:,0], bins = np.linspace(0.3, 0.55, 20))
plt.title('SNPE')
plt.ylabel('counts')
plt.plot(mu_true*np.ones(2), [0, 14], 'r', linewidth=2)
plt.xlabel('posterior mean')
plt.subplot(2,2,4)
plt.hist(out_snpe[:,1], bins = np.linspace(0.4, 0.65, 20))
plt.plot(sig_true*np.ones(2), [0, 14], 'r', linewidth=2)
plt.title('SNPE')
plt.xlabel('posterior variance')

#plt.savefig('1d_toy_case_50runs_SNPE_and_CDELFI.pdf')

plt.show()

In [ ]:
r=4
iws = trn_data[r][2]
(iws.min(), iws.max(), iws.mean())

In [ ]:
for r in range(1, len(trn_data)):
    plt.hist(trn_data[r][2])
    plt.show()